In [56]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files='data_small_train.csv')

#dataset = load_dataset(train_dc_2)

Extracting data files: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 19.24it/s]


Dataset csv downloaded and prepared to C:/Users/Alberto Navarro/.cache/huggingface/datasets/csv/default-6a4239bfa2cf3f73/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 17.85it/s]


In [57]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'TARGET', 'FECHA_STRING', 'COMENTARIO', 'COMENTARIO_LIMPIO', 'PALABRAS', 'TOKENS', 'NUM_TOKENS', 'sentimiento', 'comentario_tarea'],
        num_rows: 30000
    })
})

In [58]:
from datasets import DatasetDict, Dataset

def train_test_split(dataset_dict: DatasetDict, test_size: float = 0.2) -> DatasetDict:
    """Split a dataset dictionary into train and test based on test size.

    Args:
        dataset_dict (DatasetDict): Input dataset dictionary.
        test_size (float, optional): Fraction of data to include in the test set. Defaults to 0.2.

    Returns:
        DatasetDict: Result dataset dictionary with the desired splitting strategy.
    """
    train_test = dataset_dict["train"].train_test_split(test_size=test_size)

    return DatasetDict({
        "train": train_test["train"],
        "test": train_test["test"]
    })


In [59]:
# Uso de la función para dividir un DatasetDict en train y test
divided_data = train_test_split(dataset, test_size=0.2)

In [273]:
dataset_train_dataset = divided_data["train"].shuffle(seed=42).select([i for i in list(range(100))])
dataset_test_dataset = divided_data["test"].shuffle(seed=42).select([i for i in list(range(10))])

Loading cached shuffled indices for dataset at C:\Users\Alberto Navarro\.cache\huggingface\datasets\csv\default-6a4239bfa2cf3f73\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-0b57310459c7b36a.arrow
Loading cached shuffled indices for dataset at C:\Users\Alberto Navarro\.cache\huggingface\datasets\csv\default-6a4239bfa2cf3f73\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-bd31e962806a74f0.arrow


In [274]:
train_ds = Dataset.from_dict({ 
    "text": dataset_train_dataset["COMENTARIO_LIMPIO"], # Campo de texto 
    "label": dataset_train_dataset["TARGET"] # Campo de sentimiento
})

test_ds = Dataset.from_dict({ 
    "text": dataset_test_dataset["COMENTARIO_LIMPIO"], # Campo de texto 
    "label": dataset_test_dataset["TARGET"] # Campo de sentimiento
})

In [275]:
train_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 100
})

### Fine tuning al modelo pre entrenado

In [276]:
from transformers import T5Tokenizer

#tokenizer = T5Tokenizer.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-small')

In [277]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=64) #padding=True
 
tokenized_train = train_ds.map(preprocess_function, batched=True)
tokenized_test = test_ds.map(preprocess_function, batched=True)

In [278]:
tokenized_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 100
})

In [279]:
tokenized_test

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10
})

In [280]:
train_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 100
})

In [281]:
test_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 10
})

In [282]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [295]:
data_collator

DataCollatorWithPadding(tokenizer=T5Tokenizer(name_or_path='google/flan-t5-small', vocab_size=32100, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', 

In [362]:
from transformers import AutoModelForSequenceClassification
#model = AutoModelForSequenceClassification.from_pretrained("t5-small", num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained("google/flan-t5-small", num_labels=2)

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-t5-small and are newly initialized: ['classification_head.dense.weight', 'classification_head.dense.bias', 'classification_head.out_proj.weight', 'classification_head.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [366]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   print(logits)
   print(labels)
   predictions = np.argmax(logits, axis=-1)
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"f1": f1}

In [335]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_f1 = load_metric("f1")
    
    # Desempaquetar los valores de la tupla
    labels = eval_pred.label_ids
    preds = eval_pred.predictions.argmax(-1)
    # Calcular el F1-score
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    
    return {"f1": f1}


In [367]:
from transformers import TrainingArguments, Trainer
 
repo_name = "finetuning-sentiment-model-3000-samples"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,#tokenized_train
   eval_dataset=tokenized_test,#tokenized_test
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [261]:
tokenized_test

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 30
})

In [323]:
train_dataset = trainer.train_dataset
eval_dataset = trainer.eval_dataset
cm = trainer.compute_metrics

In [324]:
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 100
})

In [328]:
eval_dataset[0]

{'text': 'Death disco amazing Smangled x',
 'label': 1,
 'input_ids': [12358,
  5025,
  32,
  1237,
  180,
  51,
  19834,
  3,
  226,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [365]:
trainer.train()

Step,Training Loss


KeyboardInterrupt: 

In [368]:
trainer.evaluate()

(array([[0.15083723, 0.48156506],
       [0.21024026, 0.56997645],
       [0.0354127 , 0.49313873],
       [0.08557307, 0.5560947 ],
       [0.17644835, 0.5209846 ],
       [0.03333851, 0.5483879 ],
       [0.12620753, 0.5867075 ],
       [0.18800125, 0.4190094 ],
       [0.16385531, 0.45390055],
       [0.13604286, 0.46245793]], dtype=float32), array([[[ 1.08796395e-02, -9.52283293e-02, -5.31933904e-02, ...,
          9.26067680e-02,  5.24565503e-02,  1.82792917e-02],
        [ 9.54191945e-03, -1.07318364e-01, -2.10455358e-01, ...,
          5.51800318e-02, -2.14922503e-01,  7.66242370e-02],
        [-9.66642201e-02, -7.85706565e-02, -1.55693069e-01, ...,
          1.65006578e-01,  4.35043946e-02, -1.27334064e-02],
        ...,
        [ 7.11672083e-02, -2.24442720e-01,  5.49522368e-03, ...,
         -3.18226278e-01, -4.66227904e-02,  1.51638128e-02],
        [ 7.11672083e-02, -2.24442720e-01,  5.49522368e-03, ...,
         -3.18226278e-01, -4.66227904e-02,  1.51638128e-02],
        [

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 10) + inhomogeneous part.

In [374]:
!pip install numpy=1.21.6

^C


ERROR: Invalid requirement: 'numpy=1.21.6'
Hint: = is not a valid operator. Did you mean == ?


### Modelo pre entrenado

In [341]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files='data_small_train.csv')

#dataset = load_dataset(train_dc_2)

Found cached dataset csv (C:/Users/Alberto Navarro/.cache/huggingface/datasets/csv/default-6a4239bfa2cf3f73/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.17it/s]


In [342]:
from datasets import DatasetDict, Dataset

def train_test_split(dataset_dict: DatasetDict, test_size: float = 0.2) -> DatasetDict:
    """Split a dataset dictionary into train and test based on test size.

    Args:
        dataset_dict (DatasetDict): Input dataset dictionary.
        test_size (float, optional): Fraction of data to include in the test set. Defaults to 0.2.

    Returns:
        DatasetDict: Result dataset dictionary with the desired splitting strategy.
    """
    train_test = dataset_dict["train"].train_test_split(test_size=test_size)

    return DatasetDict({
        "train": train_test["train"],
        "test": train_test["test"]
    })


In [343]:
# Uso de la función para dividir un DatasetDict en train y test
divided_data = train_test_split(dataset, test_size=0.2)

Loading cached split indices for dataset at C:\Users\Alberto Navarro\.cache\huggingface\datasets\csv\default-6a4239bfa2cf3f73\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-acf047cc63c258dd.arrow and C:\Users\Alberto Navarro\.cache\huggingface\datasets\csv\default-6a4239bfa2cf3f73\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-d9743f5f584f4349.arrow


In [344]:
dataset_train_dataset = divided_data["train"].shuffle(seed=42).select([i for i in list(range(100))])
dataset_test_dataset = divided_data["test"].shuffle(seed=42).select([i for i in list(range(10))])

Loading cached shuffled indices for dataset at C:\Users\Alberto Navarro\.cache\huggingface\datasets\csv\default-6a4239bfa2cf3f73\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-0b57310459c7b36a.arrow
Loading cached shuffled indices for dataset at C:\Users\Alberto Navarro\.cache\huggingface\datasets\csv\default-6a4239bfa2cf3f73\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-bd31e962806a74f0.arrow


In [345]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=64) #padding=True
 
tokenized_train = train_ds.map(preprocess_function, batched=True)
tokenized_test = test_ds.map(preprocess_function, batched=True)

In [228]:
texts_with_labels = []

for i in range(len(tokenized_train)):
    text = tokenized_train[i]["text"]
    label = tokenized_train[i]["label"]
    texts_with_labels.append({"text": text, "sentiment": label})


In [346]:
texts_with_labels

[{'text': 'icg979 hows your summer so far ', 'sentiment': 1},
 {'text': 'Happy Mothers Day to All the moms and to Those that have Moms That means Happy Mothers Day EVeryONe ',
  'sentiment': 1},
 {'text': 'Hes Just Not That Into You Hate ', 'sentiment': 0},
 {'text': 'The queen has left the building I did some cleaning Ironing is done now too ',
  'sentiment': 1},
 {'text': 'why I can hear its quotjustquot 86ÂF noooo its 915ÂF ',
  'sentiment': 0},
 {'text': 'IAmAlexSuavez i added sumthing ', 'sentiment': 1},
 {'text': 'One of my new friends just left ', 'sentiment': 0},
 {'text': 'Good morning ', 'sentiment': 1},
 {'text': 'Up early because of the time difference Going for a walk with my new Canon 450D ',
  'sentiment': 1},
 {'text': 'I just got flowers ', 'sentiment': 1},
 {'text': '15 days to golast week of work ', 'sentiment': 0},
 {'text': 'Stay tuned for more info on this years performers You are in for a pleasant surprise ',
  'sentiment': 1},
 {'text': 'westhgate Shame about yo

In [243]:
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

model = AutoModelWithLMHead.from_pretrained("google/flan-t5-small")

def get_sentiment_list(texts_with_labels):
    predicted_sentiments = []  # Lista para almacenar las etiquetas de sentimiento generadas por el modelo
    true_sentiments = []  # Lista para almacenar las etiquetas de sentimiento reales

    for item in texts_with_labels:
        text = item["text"]
        true_sentiment = item["sentiment"]

        # Generar etiquetas de sentimiento (puedes mantener tu código existente aquí)
        input_ids = tokenizer.encode('sentiment: ' + text + '</s>', return_tensors='pt')
        output = model.generate(input_ids=input_ids, max_length=2)
        dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in output]
        predicted_sentiment = dec[0]
        
        predicted_sentiment = 1 if predicted_sentiment == "positive" else 0

        predicted_sentiments.append(predicted_sentiment)
        true_sentiments.append(true_sentiment)

    return predicted_sentiments, true_sentiments

predicted_sentiments, true_sentiments = get_sentiment_list(texts_with_labels)

# Calcula el F1-score para comparar las etiquetas reales con las predichas
f1 = f1_score(true_sentiments, predicted_sentiments, average='binary')

print("F1-score:", f1)

C:\Users\Alberto Navarro\anaconda3\envs\env2\Lib\site-packages\transformers\models\auto\modeling_auto.py:1468: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


F1-score: 0.8168168168168167
